In [1]:
!export PYTHONPATH=/home/manishav/addons/site-packages/:/home/manishav/

In [2]:
import matchzoo as mz
import numpy as np
import matplotlib.pyplot as plt


Using TensorFlow backend.


##### Fetch the dataset

In [ ]:
#!wget https://msmarco.blob.core.windows.net/msmarcoranking/qrels.train.tsv

In [ ]:
#!wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz

In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
##### Load the embeddings
#!unzip glove.6B.zip

In [3]:
import matchzoo as mz
import pandas as pd

from sklearn.utils import shuffle
print(mz.__version__)
from scipy.spatial.distance import cosine
from matchzoo.metrics import Precision


2.2.0


##### Define functions and metrics


In [4]:
"""Precision for ranking."""
import numpy as np

from matchzoo.engine.base_metric import BaseMetric, sort_and_couple


### Add 

class PrecisionCheck(BaseMetric):
    """Precision metric."""

    ALIAS = 'precision'

    def __init__(self, k: int = 1, threshold: float = 0.):
        """
        :class:`PrecisionMetric` constructor.
        :param k: Number of results to consider.
        :param threshold: the label threshold of relevance degree.
        """
        self._k = k
        self._threshold = threshold

    def __repr__(self) -> str:
        """:return: Formated string representation of the metric."""
        return f"{self.ALIAS}@{self._k}({self._threshold})"

    def __call__(self, y_true: np.array, y_pred: np.array) -> float:
        #print('number of results', y_true.shape)
        """
        Calculate precision@k.
        Example:
            >>> y_true = [0, 0, 0, 1]
            >>> y_pred = [0.2, 0.4, 0.3, 0.1]
            >>> Precision(k=1)(y_true, y_pred)
            0.0
            >>> Precision(k=2)(y_true, y_pred)
            0.0
            >>> Precision(k=4)(y_true, y_pred)
            0.25
            >>> Precision(k=5)(y_true, y_pred)
            0.2
        :param y_true: The ground true label of each document.
        :param y_pred: The predicted scores of each document.
        :return: Precision @ k
        :raises: ValueError: len(r) must be >= k.
        """
        if self._k <= 0:
            raise ValueError(f"k must be greater than 0."
                             f"{self._k} received.")
        # sorted list of pairs.
        coupled_pair = sort_and_couple(y_true, y_pred)
        
        precision = 0.0
        
        for idx, (label, score) in enumerate(coupled_pair):
            
            #if idx < 10:
            #    print('[',label, score, ']', )
            if idx >= self._k:
                break
            if label > self._threshold:
                precision += 1.
        return precision / self._k
    
    
"""Recall for ranking."""
import numpy as np

from matchzoo.engine.base_metric import BaseMetric, sort_and_couple


class RecallCheck(BaseMetric):
    """Recall metric."""

    ALIAS = 'recall'

    def __init__(self, k: int = 1, threshold: float = 0.):
        """
        :class:`RecallMetric` constructor.
        :param k: Number of results to consider.
        :param threshold: the label threshold of relevance degree.
        """
        self._k = k
        self._threshold = threshold

    def __repr__(self) -> str:
        """:return: Formated string representation of the metric."""
        return f"{self.ALIAS}@{self._k}({self._threshold})"

    def __call__(self, y_true: np.array, y_pred: np.array) -> float:
        if self._k == 5:
            print('number of results', y_true.shape, (y_true > self._threshold ).sum())
        """
        Calculate Recall@k.
        Example:
            >>> y_true = [0, 0, 0, 1]
            >>> y_pred = [0.2, 0.4, 0.3, 0.1]
            >>> Recall(k=1)(y_true, y_pred)
            0.0
            >>> Recall(k=2)(y_true, y_pred)
            0.0
            >>> Recall(k=4)(y_true, y_pred)
            1.0
            >>> Recall(k=5)(y_true, y_pred)
            1.0
        :param y_true: The ground true label of each document.
        :param y_pred: The predicted scores of each document.
        :return: Recall @ k
        :raises: ValueError: len(r) must be >= k.
        """
        if self._k <= 0:
            raise ValueError(f"k must be greater than 0."
                             f"{self._k} received.")
        # sorted list of pairs.
        coupled_pair = sort_and_couple(y_true, y_pred)
        
        recall = 0.0
        rel_docs = (y_true > self._threshold ).sum() * 1.0
        for idx, (label, score) in enumerate(coupled_pair):
            
            if idx < 5 and self._k == 5:
                print('[',label, score, ']', )
            if idx >= self._k:
                break
            if label > self._threshold:
                recall += 1.
        return recall / rel_docs

In [5]:

"""Matchzoo toolkit for token embedding."""

import csv
import typing

import numpy as np
import pandas as pd


class Embedding(object):
    """
    Embedding class.
    Examples::
        >>> import matchzoo as mz
        >>> train_raw = mz.datasets.toy.load_data()
        >>> pp = mz.preprocessors.NaivePreprocessor()
        >>> train = pp.fit_transform(train_raw, verbose=0)
        >>> vocab_unit = mz.build_vocab_unit(train, verbose=0)
        >>> term_index = vocab_unit.state['term_index']
        >>> embed_path = mz.datasets.embeddings.EMBED_RANK
    To load from a file:
        >>> embedding = mz.embedding.load_from_file(embed_path)
        >>> matrix = embedding.build_matrix(term_index)
        >>> matrix.shape[0] == len(term_index) + 1
        True
    To build your own:
        >>> data = pd.DataFrame(data=[[0, 1], [2, 3]], index=['A', 'B'])
        >>> embedding = mz.Embedding(data)
        >>> matrix = embedding.build_matrix({'A': 2, 'B': 1})
        >>> matrix.shape == (3, 2)
        True
    """

    def __init__(self, data: pd.DataFrame):
        """
        Embedding.
        :param data: DataFrame to use as term to vector mapping.
        """
        self._data = data

    @property
    def input_dim(self) -> int:
        """:return Embedding input dimension."""
        return self._data.shape[0]

    @property
    def output_dim(self) -> int:
        """:return Embedding output dimension."""
        return self._data.shape[1]

    def build_matrix(
        self,
        term_index: typing.Union[
            dict, mz.preprocessors.units.Vocabulary.TermIndex],
        initializer=lambda: np.random.uniform(-0.2, 0.2)
    ) -> np.ndarray:
        """
        Build a matrix using `term_index`.
        :param term_index: A `dict` or `TermIndex` to build with.
        :param initializer: A callable that returns a default value for missing
            terms in data. (default: a random uniform distribution in range)
            `(-0.2, 0.2)`).
        :return: A matrix.
        """
        input_dim = len(term_index) + 1
        
        print('Embedding to matrix, input & output', input_dim, self.output_dim)
        
        matrix = np.empty((input_dim, self.output_dim))
        for index in np.ndindex(*matrix.shape):
            matrix[index] = initializer()

        #valid_keys = set(self._data.index)
        terms_to_find = term_index.keys()
        valid_data_frame = self._data[self._data.index[-len(self._data):].isin(terms_to_find)]
        
        nf_count = 0
        total = 0
        
        for term, values in valid_data_frame.iterrows():
            if total % 50000==0:
                print('Words completed', total)
            total +=1
                
            try:
                matrix[term_index[term]] = values
            except Exception as ex:
                nf_count+=1
                print('Word not found', term)
                    
        print('Words not found in embedding ', nf_count)
        
        return matrix


def load_from_file(file_path: str, mode: str = 'word2vec') -> Embedding:
    """
    Load embedding from `file_path`.
    :param file_path: Path to file.
    :param mode: Embedding file format mode, one of 'word2vec' or 'glove'.
        (default: 'word2vec')
    :return: An :class:`matchzoo.embedding.Embedding` instance.
    """
    if mode == 'word2vec':
        data = pd.read_csv(file_path,
                           sep=" ",
                           index_col=0,
                           header=None,
                           skiprows=1)
        print(data.head())
    elif mode == 'glove':
        data = pd.read_csv(file_path,
                           sep=" ",
                           index_col=0,
                           header=None,
                           quoting=csv.QUOTE_NONE)
    else:
        raise TypeError(f"{mode} is not a supported embedding type."
                        f"`word2vec` or `glove` expected.")
    return Embedding(data)





In [6]:
import ast
slist = ast.literal_eval(open('msmarco_data/long_stop_words','r').read())

stop_words = list( set(['are', 'get','you','dont','want','take','have','need','let','your','their','theirs',\
              'still','these','that','could','should', 'would', 'with','does','this','used','make',\
              'makes','made','takes','take','those','when','without','more','becasue','there','aren',\
              'keep','seem','seems','wont','shouldn','shouldnt','only','than','know','every','also',\
              'brand','become','most','other','others','meant','thing','things','happens','anything',\
              'gets','sets','both','bring','then','goes','some','someone','see','article','redirect',\
              'sent','into','about','what','where','give','going','like','look','looks','having','other',\
              'gives','give','given','uses','used','through','though','very','doesn','many','even','mine',\
              'myself','always', 'self','currently','along','else','comes','come','came','likes','like',\
              'because','can','the', 'an', 'to', 'and', 'from', 'for', 'we', 'you', 'i', 'so','such',\
               'a', 'at', 'b', 'be', 'in', 'of', 'on', 'was', 'is','been','while','will','they','them']) | \
                set(slist))
print(len(stop_words))

193


In [ ]:
#! tar -xvzf msmarco_data/collectionandqueries.tar.gz

##### Define paths

In [7]:
path='/home/manishav/irexplain/src/main/python/deep_explain/msmarco_data/'
collection_path = path+ 'collection/'
embedding_path =  path+ 'embedding/'

#### Load queries

In [8]:
import re
query_list = []
match_words = r'define|what|when'

for line in open(collection_path+'queries.train.tsv','r'):
    split = line.strip().split('\t')
    
    match = re.search(match_words,split[1])
    if not match:
        query_list.append({'qid':split[0], 'query': split[1].strip()})

In [9]:
query_frame = pd.DataFrame(query_list)

In [10]:
query_frame.head()

,qid,query
0,510633,tattoo fixers how much does it cost
1,278900,how many cars enter the la jolla concours d' e...
2,303205,how much can i contribute to nondeductible ira
3,492875,sanitizer temperature
4,54528,blood clots in urine after menopause


In [11]:
query_frame.shape

(478643, 2)

#### load qrels

In [12]:
qrels = []
for line in open(collection_path+'qrels.train.tsv','r'):
    split = line.strip().split('\t')
    qrels.append({'qid':split[0], 'pid': split[2], 'rel':split[3] })
qrel_frame = pd.DataFrame(qrels) 

In [13]:
qrel_frame.head()

,qid,pid,rel
0,1185869,0,1
1,1185868,16,1
2,597651,49,1
3,403613,60,1
4,1183785,389,1


In [14]:
#! wget https://msmarco.blob.core.windows.net/msmarcoranking/qidpidtriples.train.full.tar.gz

In [15]:
neg_rows = []
query_count = {}
count = 0
for line in open('qidpidtriples.train.full.tar','r'):
    split = line.split('\t')
    
    qid = split[0]
    
    if qid not in query_count:
        query_count[qid] = 0
    
    if query_count[qid]<30:
        neg_rows.append({'qid': qid, 'pid':  split[2].strip(), 'rel':0})
        query_count[qid]+=1
        count+=1 

    if count % 100000 == 0:
        print(count)
        
    if count > 1000000:
        break

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000


In [16]:
neg_frame = pd.DataFrame(neg_rows)

In [17]:
qrel_frame = pd.concat([qrel_frame, neg_frame])
qrel_frame.drop_duplicates(inplace=True)
qrel_frame.head()

,qid,pid,rel
0,1185869,0,1
1,1185868,16,1
2,597651,49,1
3,403613,60,1
4,1183785,389,1


In [18]:
qrel_frame.tail()

,qid,pid,rel
999996,916865,5790524,0
999997,1148964,523094,0
999998,1015913,4524379,0
999999,563466,523094,0
1000000,399782,5790524,0


In [19]:
qid_pid_rel_frame = pd.merge(query_frame, qrel_frame, left_on='qid', right_on='qid', suffixes=('', '_y'))

In [20]:
qid_pid_rel_frame.shape

(870588, 4)

In [21]:
qid_pid_rel_frame['rel'].value_counts()

0    574857
1    295731
Name: rel, dtype: int64

In [23]:
qid_pid_rel_frame['rel'] = qid_pid_rel_frame['rel'].apply(int)

In [24]:
qrel_frame[qrel_frame['qid'].isin(query_frame['qid'].tolist())].shape

(870588, 3)

##### Load documents 

In [25]:
pid_set = set(qid_pid_rel_frame['pid'].tolist())

paragraphs = []
for line in open(collection_path+'collection.tsv','r'):
    split = line.strip().split('\t')
    if split[0] in pid_set:
        paragraphs.append({'pid': split[0],\
                           'paragraph': split[1].strip()})
        
para_frame = pd.DataFrame(paragraphs) 

In [26]:
para_frame.shape

(467480, 2)

In [27]:
qid_pid_para_rel_frame = pd.merge(qid_pid_rel_frame, para_frame, left_on='pid', right_on='pid', suffixes=('', '_y'))

In [28]:
qid_pid_para_rel_frame.head()

,qid,query,pid,rel,paragraph
0,510633,tattoo fixers how much does it cost,1879754,1,1 Most tattoo artists charge an hourly rate th...
1,303205,how much can i contribute to nondeductible ira,6487240,1,Nondeductible IRA Contributions. For a traditi...
2,303205,how much can i contribute to nondeductible ira,821461,0,Just because you can contribute to a 401(k) pl...
3,11658,adjusted gross income definition,821461,0,Just because you can contribute to a 401(k) pl...
4,68667,can i contribute to a roth ira without earned ...,821461,0,Just because you can contribute to a 401(k) pl...


In [29]:
del(para_frame)

In [30]:
qid_pid_para_rel_frame[qid_pid_para_rel_frame['rel'] == '1'].shape

/home/manishav/addons/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


(0, 5)

#### Prepare train and test_frame for the model training

In [31]:
from sklearn.utils import shuffle
import random 
# sample 10K queries
query_list = qid_pid_para_rel_frame['qid'].drop_duplicates().tolist()

train_queries = random.sample(query_list, 50000)
test_queries = random.sample(list(set(query_list) - set(train_queries)), 5000)

final_train_frame = shuffle(qid_pid_para_rel_frame[qid_pid_para_rel_frame['qid'].isin(train_queries)][['query','paragraph','rel']])

In [32]:
final_train_frame = final_train_frame.reset_index(drop=True)
final_train_frame.columns=['text_left','text_right','label']

In [33]:
final_train_frame.head()

,text_left,text_right,label
0,should you fertilize tomato plants in pots rig...,"Mature Plants. In the mature stage of growth, ...",0
1,how many ml are in one gallon,"1 US gallon of water (gal) = 3,785.41 millilit...",1
2,is it the acl or mcl on the outside of the knee,A new total knee replacement that saves all of...,0
3,where do goldenrod gall flies live,Kentucky State Flag - Information. 1 The flag...,0
4,anytime fitness price per month,Anytime Fitness Franchise Reviews. Anytime Fit...,0


In [34]:
### sample queries for test (should not overlap with train)
final_test_frame = shuffle(qid_pid_para_rel_frame[qid_pid_para_rel_frame['qid'].isin(test_queries)]\
                            [['query','paragraph','rel']])

In [35]:
final_test_frame = final_test_frame.reset_index(drop=True)
final_test_frame.columns=['text_left','text_right','label']
final_test_frame.head()

,text_left,text_right,label
0,who owns energizer batteries,A wet cell is a cell that contains a liquid el...,0
1,how much does a space mission to mars cost,12:52PM BST 12 Sep 2015. An astronaut has brok...,0
2,foil definition drama term,"In fiction, a foil is a character who contrast...",1
3,how many pieces in domino game,Quick Answer. There are a total of 28 dominoes...,1
4,which bases are in dna,Indel is a molecular biology term for the inse...,0


In [36]:
train_raw =  mz.pack(final_train_frame)# mz.datasets.toy.load_data(stage='train', task=task)
test_raw =  mz.pack(final_test_frame)#mz.datasets.toy.load_data(stage='test', task=task)

In [37]:
train_raw.left.head()

,text_left
id_left,
L-0,should you fertilize tomato plants in pots rig...
L-1,how many ml are in one gallon
L-2,is it the acl or mcl on the outside of the knee
L-3,where do goldenrod gall flies live
L-4,anytime fitness price per month


#### Preprocess the text

In [38]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=15,
             fixed_length_right=30,
             filter_mode='idf',
             filter_low_freq=2,
             filter_high_freq=1000, remove_stop_words=True)


In [39]:
preprocessor.fit(train_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 50000/50000 [00:15<00:00, 3207.56it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 123376/123376 [03:12<00:00, 641.89it/s]
Building Vocabulary from a datapack.: 100%|██████████| 4466636/4466636 [00:02<00:00, 1767772.22it/s]


In [40]:
len(preprocessor.context['vocab_unit'].state['term_index'])

200793

In [41]:
train_processed = preprocessor.transform(train_raw,verbose=0)
test_processed = preprocessor.transform(test_raw,verbose=0)

In [42]:
vocab_unit = preprocessor.context['vocab_unit']
print('Orig Text:', train_processed.left.loc['L-0']['text_left'])
sequence = train_processed.left.loc['L-0']['text_left']
print('Transformed Indices:', sequence)
print('Transformed Indices Meaning:',
      '_'.join([vocab_unit.state['index_term'][i] for i in sequence]))

Orig Text: [165749, 153389, 188624, 44092, 91904, 166807, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Transformed Indices: [165749, 153389, 188624, 44092, 91904, 166807, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Transformed Indices Meaning: fertilize_tomato_plants_pots_right_planting_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>


##### Define the ranking task

In [43]:

task = mz.tasks.Ranking()
task.metrics = [PrecisionCheck(k=3, threshold=0),
    PrecisionCheck(k=5, threshold=0),
    RecallCheck(k=3, threshold=0),
    RecallCheck(k=5, threshold=0),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3, threshold=0),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5, threshold=0),
]
print(task)

Ranking Task


In [ ]:
! ls {embedding_path}

In [44]:
#train_generator = mz.PairDataGenerator(train_processed, num_dup=1, num_neg=4, batch_size=32, shuffle=True)
#len(train_generator)
word_embeddings = load_from_file(embedding_path+'glove.6B.50d.txt', mode= 'glove')

In [45]:
#history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=5, use_multiprocessing=False)

In [46]:
model = mz.models.DRMMTKS()
model.params['task'] = task
model.params['embedding_input_dim'] =  len(preprocessor.context['vocab_unit'].state['term_index']) +1 
model.params['embedding_output_dim'] = word_embeddings.output_dim
#model.params['embedding_output_dim'] = 11

model.params['top_k'] = 10
model.params['mlp_num_layers'] = 2
model.params['mlp_num_units'] = 20
model.params['mlp_num_fan_out'] = 10
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = 'adadelta'
#model.params['mlp_num_units'] = mlp_units

model.params['embedding_trainable'] = True

model.guess_and_fill_missing_params(verbose=1)

model.params.update(preprocessor.context)
print('Model param',model.params)

model.build()
#model.load_embedding_matrix(embedding_matrix)
model.compile()
print('Model complete',model.params.completed())
model.backend.summary()

Model param model_class                   <class 'matchzoo.models.drmmtks.DRMMTKS'>
input_shapes                  [(15,), (30,)]
task                          Ranking Task
optimizer                     adadelta
with_embedding                True
embedding_input_dim           200793
embedding_output_dim          50
embedding_trainable           True
with_multi_layer_perceptron   True
mlp_num_units                 20
mlp_num_layers                2
mlp_num_fan_out               10
mlp_activation_func           tanh
mask_value                    -1
top_k                         10
Model complete True
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 15)           0                                            
_________________________________________________________________________________

In [47]:
x, y = train_processed.unpack()
test_x, test_y = test_processed.unpack()

In [48]:
model.fit(x, y, batch_size=1000, epochs=7)

/home/y/var/python36/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
155887/155887 [==============================] - 38s 245us/step - loss: 0.2221
Epoch 2/7
155887/155887 [==============================] - 36s 230us/step - loss: 0.2154
Epoch 3/7
155887/155887 [==============================] - 36s 231us/step - loss: 0.2061
Epoch 4/7
155887/155887 [==============================] - 36s 232us/step - loss: 0.1907
Epoch 5/7
155887/155887 [==============================] - 36s 230us/step - loss: 0.1780
Epoch 6/7
155887/155887 [==============================] - 36s 231us/step - loss: 0.1607
Epoch 7/7
155887/155887 [==============================] - 36s 234us/step - loss: 0.1393


In [49]:
pred_x, pred_y = test_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))

In [ ]:
?evaluate

In [50]:
model.evaluate(test_x, test_y )

number of results (5,) 1
[ 1.0 0.18990422785282135 ]
[ 0.0 0.1835498958826065 ]
[ 0.0 0.15567804872989655 ]
[ 0.0 0.10907791554927826 ]
[ 0.0 -0.09946700930595398 ]
number of results (5,) 1
[ 1.0 0.6556262373924255 ]
[ 0.0 0.30056846141815186 ]
[ 0.0 0.2826955318450928 ]
[ 0.0 0.10961006581783295 ]
[ 0.0 -0.3542860746383667 ]
number of results (5,) 1
[ 0.0 0.9411147236824036 ]
[ 1.0 0.8572648167610168 ]
[ 0.0 0.5517998337745667 ]
[ 0.0 0.3176901936531067 ]
[ 0.0 -0.25784483551979065 ]
number of results (4,) 1
[ 1.0 0.3889808654785156 ]
[ 0.0 0.31211984157562256 ]
[ 0.0 0.23594743013381958 ]
[ 0.0 0.003933504223823547 ]
number of results (1,) 1
[ 1.0 0.8693211674690247 ]
number of results (5,) 1
[ 0.0 0.7872974276542664 ]
[ 1.0 0.7338436841964722 ]
[ 0.0 0.5918559432029724 ]
[ 0.0 0.5709472298622131 ]
[ 0.0 0.47387921810150146 ]
number of results (4,) 1
[ 1.0 0.5924009680747986 ]
[ 0.0 0.261926531791687 ]
[ 0.0 0.09526820480823517 ]
[ 0.0 -0.15294812619686127 ]
number of results (8,) 2


number of results (7,) 1
[ 1.0 0.7361119985580444 ]
[ 0.0 0.4449988603591919 ]
[ 0.0 0.423575222492218 ]
[ 0.0 0.2795533537864685 ]
[ 0.0 0.27413415908813477 ]
number of results (7,) 1
[ 0.0 0.7402163147926331 ]
[ 1.0 0.707940936088562 ]
[ 0.0 0.4245451092720032 ]
[ 0.0 0.30414289236068726 ]
[ 0.0 0.1414341777563095 ]
number of results (4,) 1
[ 1.0 0.6494148969650269 ]
[ 0.0 0.5082332491874695 ]
[ 0.0 0.45710158348083496 ]
[ 0.0 0.04370418190956116 ]
number of results (6,) 1
[ 1.0 0.7562926411628723 ]
[ 0.0 0.5274472236633301 ]
[ 0.0 0.493119478225708 ]
[ 0.0 0.311900794506073 ]
[ 0.0 0.25658804178237915 ]
number of results (5,) 1
[ 1.0 0.6640215516090393 ]
[ 0.0 0.5636869072914124 ]
[ 0.0 0.33411741256713867 ]
[ 0.0 0.24517203867435455 ]
[ 0.0 -0.08299575746059418 ]
number of results (4,) 1
[ 1.0 0.5362440943717957 ]
[ 0.0 0.4888165593147278 ]
[ 0.0 0.2805001735687256 ]
[ 0.0 0.13235841691493988 ]
number of results (4,) 1
[ 1.0 0.8828884363174438 ]
[ 0.0 0.7779388427734375 ]
[ 0.0 0.4

[ 1.0 0.6913090348243713 ]
[ 1.0 0.6851720809936523 ]
[ 0.0 0.6367203593254089 ]
[ 0.0 0.5976060032844543 ]
number of results (5,) 1
[ 0.0 0.688511312007904 ]
[ 1.0 0.6089723706245422 ]
[ 0.0 0.2880275249481201 ]
[ 0.0 0.15687520802021027 ]
[ 0.0 0.06299270689487457 ]
number of results (2,) 1
[ 1.0 0.6468836665153503 ]
[ 0.0 0.3843204975128174 ]
number of results (7,) 1
[ 1.0 0.8403241038322449 ]
[ 0.0 0.5483848452568054 ]
[ 0.0 0.4470386505126953 ]
[ 0.0 0.19501221179962158 ]
[ 0.0 0.021095797419548035 ]
number of results (1,) 1
[ 1.0 0.6688717603683472 ]
number of results (4,) 1
[ 1.0 0.7434741258621216 ]
[ 0.0 0.7107427716255188 ]
[ 0.0 0.6121344566345215 ]
[ 0.0 0.17393076419830322 ]
number of results (13,) 2
[ 0.0 0.8254494667053223 ]
[ 0.0 0.7865768671035767 ]
[ 1.0 0.6679397225379944 ]
[ 0.0 0.6650967597961426 ]
[ 1.0 0.643807590007782 ]
number of results (4,) 1
[ 0.0 0.7390244007110596 ]
[ 1.0 0.6247147917747498 ]
[ 0.0 0.33431780338287354 ]
[ 0.0 0.28434425592422485 ]
number o

[ 1.0 0.825506865978241 ]
[ 0.0 0.6376462578773499 ]
[ 0.0 0.5920918583869934 ]
[ 0.0 0.23544925451278687 ]
[ 0.0 -0.0820295587182045 ]
number of results (8,) 1
[ 1.0 0.45220547914505005 ]
[ 0.0 0.16708619892597198 ]
[ 0.0 0.09937705099582672 ]
[ 0.0 0.04815508425235748 ]
[ 0.0 0.0393746942281723 ]
number of results (2,) 1
[ 0.0 0.5382988452911377 ]
[ 1.0 0.3618670701980591 ]
number of results (5,) 1
[ 1.0 0.6475618481636047 ]
[ 0.0 0.23163940012454987 ]
[ 0.0 0.21720720827579498 ]
[ 0.0 0.13705040514469147 ]
[ 0.0 0.005696490406990051 ]
number of results (5,) 1
[ 1.0 0.6351764798164368 ]
[ 0.0 0.4218711256980896 ]
[ 0.0 0.4134114384651184 ]
[ 0.0 0.3010852336883545 ]
[ 0.0 0.012321971356868744 ]
number of results (5,) 1
[ 1.0 0.9161729216575623 ]
[ 0.0 0.6307510137557983 ]
[ 0.0 0.4522002339363098 ]
[ 0.0 0.2684822082519531 ]
[ 0.0 0.12383340299129486 ]
number of results (4,) 1
[ 0.0 0.6147515177726746 ]
[ 1.0 0.5641562342643738 ]
[ 0.0 0.22453247010707855 ]
[ 0.0 -0.01582752168178558

[ 0.0 0.40044254064559937 ]
[ 1.0 0.3585048317909241 ]
[ 0.0 0.346260130405426 ]
[ 0.0 0.20747442543506622 ]
[ 0.0 0.17220337688922882 ]
number of results (3,) 1
[ 1.0 0.7642118334770203 ]
[ 0.0 0.5121637582778931 ]
[ 0.0 0.09057967364788055 ]
number of results (3,) 1
[ 0.0 0.8381403684616089 ]
[ 0.0 0.7287629842758179 ]
[ 1.0 0.7223991751670837 ]
number of results (1,) 1
[ 1.0 0.49448245763778687 ]
number of results (3,) 1
[ 0.0 0.8771541714668274 ]
[ 0.0 0.6245743632316589 ]
[ 1.0 0.5632100105285645 ]
number of results (8,) 1
[ 0.0 0.8299995064735413 ]
[ 0.0 0.18197424709796906 ]
[ 0.0 0.11242802441120148 ]
[ 1.0 0.08277232944965363 ]
[ 0.0 -0.03409425914287567 ]
number of results (6,) 1
[ 0.0 0.7480457425117493 ]
[ 0.0 0.7116288542747498 ]
[ 0.0 0.6518319845199585 ]
[ 1.0 0.5718318223953247 ]
[ 0.0 0.4209308624267578 ]
number of results (1,) 1
[ 1.0 0.30963557958602905 ]
number of results (3,) 1
[ 0.0 0.5174292325973511 ]
[ 1.0 0.3754498362541199 ]
[ 0.0 0.24285505712032318 ]
number

...

[ 1.0 0.5557066202163696 ]
[ 0.0 0.48754411935806274 ]
[ 0.0 0.026128128170967102 ]
[ 0.0 -0.2775292992591858 ]
[ 0.0 -0.29936808347702026 ]
number of results (4,) 1
[ 1.0 0.5897349715232849 ]
[ 0.0 0.19067208468914032 ]
[ 0.0 -0.09466545283794403 ]
[ 0.0 -0.18331323564052582 ]
number of results (8,) 2
[ 1.0 0.840709388256073 ]
[ 1.0 0.7306944727897644 ]
[ 0.0 0.6001539826393127 ]
[ 0.0 0.5273179411888123 ]
[ 0.0 0.45195472240448 ]
number of results (2,) 1
[ 1.0 0.5990361571311951 ]
[ 0.0 0.25185608863830566 ]
number of results (5,) 1
[ 1.0 0.7478192448616028 ]
[ 0.0 0.7126354575157166 ]
[ 0.0 0.6192323565483093 ]
[ 0.0 0.31388038396835327 ]
[ 0.0 0.07505150139331818 ]
number of results (9,) 2
[ 1.0 0.6577731370925903 ]
[ 0.0 0.5524512529373169 ]
[ 0.0 0.5157719254493713 ]
[ 0.0 0.4610614776611328 ]
[ 0.0 0.36093848943710327 ]
number of results (2,) 1
[ 1.0 0.5874159932136536 ]
[ 0.0 0.5441117286682129 ]
number of results (9,) 2
[ 0.0 0.7727935910224915 ]
[ 1.0 0.5849503874778748 ]
[ 0

[ 0.0 0.8682035207748413 ]
[ 1.0 0.7744613289833069 ]
[ 0.0 0.4891830086708069 ]
[ 0.0 0.4674358367919922 ]
[ 0.0 0.4644520878791809 ]
number of results (4,) 1
[ 0.0 0.4228031635284424 ]
[ 0.0 0.3622394800186157 ]
[ 1.0 0.31069207191467285 ]
[ 0.0 0.2789352536201477 ]
number of results (4,) 1
[ 0.0 0.5668399333953857 ]
[ 1.0 0.4504888653755188 ]
[ 0.0 0.055186644196510315 ]
[ 0.0 -0.011286452412605286 ]
number of results (4,) 1
[ 1.0 0.8884353041648865 ]
[ 0.0 0.4618251919746399 ]
[ 0.0 0.22950555384159088 ]
[ 0.0 0.13840819895267487 ]
number of results (1,) 1
[ 1.0 0.7323675155639648 ]
number of results (5,) 1
[ 1.0 0.7864087224006653 ]
[ 0.0 0.6356557011604309 ]
[ 0.0 0.6045637130737305 ]
[ 0.0 0.5062981247901917 ]
[ 0.0 0.45904386043548584 ]
number of results (6,) 1
[ 0.0 0.9372043013572693 ]
[ 1.0 0.9174804091453552 ]
[ 0.0 0.6529543995857239 ]
[ 0.0 0.64973384141922 ]
[ 0.0 0.3047311305999756 ]
number of results (4,) 1
[ 1.0 0.6208702325820923 ]
[ 0.0 0.27321022748947144 ]
[ 0.0 0

[ 0.0 0.7280224561691284 ]
[ 0.0 0.6640149354934692 ]
[ 1.0 0.5254266858100891 ]
[ 0.0 -0.015660129487514496 ]
[ 0.0 -0.09318223595619202 ]
number of results (3,) 1
[ 1.0 0.49247443675994873 ]
[ 0.0 0.26319390535354614 ]
[ 0.0 0.16792987287044525 ]
number of results (1,) 1
[ 1.0 0.5441775918006897 ]
number of results (6,) 1
[ 1.0 0.6745866537094116 ]
[ 0.0 0.5172916054725647 ]
[ 0.0 0.32240182161331177 ]
[ 0.0 0.30389535427093506 ]
[ 0.0 0.2633252739906311 ]
number of results (3,) 1
[ 1.0 0.7753019332885742 ]
[ 0.0 0.266526460647583 ]
[ 0.0 0.18209697306156158 ]
number of results (5,) 1
[ 0.0 0.5287998914718628 ]
[ 0.0 0.3481605648994446 ]
[ 0.0 0.3408006429672241 ]
[ 1.0 0.3136388063430786 ]
[ 0.0 0.28697293996810913 ]
number of results (2,) 2
[ 1.0 0.6488199234008789 ]
[ 1.0 0.26700252294540405 ]
number of results (5,) 1
[ 0.0 0.7422104477882385 ]
[ 0.0 0.31120336055755615 ]
[ 0.0 0.12913301587104797 ]
[ 0.0 0.01776137948036194 ]
[ 1.0 0.007711872458457947 ]
number of results (7,) 1


number of results (1,) 1
[ 1.0 0.7124558091163635 ]
number of results (6,) 1
[ 1.0 0.48348522186279297 ]
[ 0.0 0.3659942150115967 ]
[ 0.0 0.11594440042972565 ]
[ 0.0 -0.034817203879356384 ]
[ 0.0 -0.06719698011875153 ]
number of results (3,) 1
[ 0.0 0.480044424533844 ]
[ 0.0 0.46707653999328613 ]
[ 1.0 0.27547895908355713 ]
number of results (5,) 1
[ 1.0 0.8195676207542419 ]
[ 0.0 0.6528603434562683 ]
[ 0.0 0.43677157163619995 ]
[ 0.0 0.2380678802728653 ]
[ 0.0 0.15706972777843475 ]
number of results (5,) 1
[ 0.0 0.37283235788345337 ]
[ 0.0 0.20591510832309723 ]
[ 1.0 0.01808936893939972 ]
[ 0.0 0.01680612564086914 ]
[ 0.0 0.011896386742591858 ]
number of results (6,) 1
[ 1.0 0.5362621545791626 ]
[ 0.0 0.47245264053344727 ]
[ 0.0 0.37956100702285767 ]
[ 0.0 0.3724544048309326 ]
[ 0.0 0.24008138477802277 ]
number of results (5,) 1
[ 1.0 0.7455385327339172 ]
[ 0.0 0.6322961449623108 ]
[ 0.0 0.5368112921714783 ]
[ 0.0 0.36279839277267456 ]
[ 0.0 0.29909664392471313 ]
number of results (6,

{precision@3(0): 0.32633333333333336,
 precision@5(0): 0.20784,
 recall@3(0): 0.9336399999999999,
 recall@5(0): 0.9846133333333333,
 normalized_discounted_cumulative_gain@3(0): 0.8530612126152634,
 normalized_discounted_cumulative_gain@5(0): 0.8749271610302636}